# Classifying Credit Card Fraud

In [1]:
try: # Wrapper to save time when running all cells
    # Standard Packages
    import pandas as pd
    import numpy as np
    import markdown
    import os

    # Viz Packages
    import seaborn as sns
    import matplotlib.pyplot as plt

    # Scipy Stats
    import scipy.stats as stats
    from scipy.special import logit, expit

    # Statsmodel Api
    import statsmodels.api as sm
    from statsmodels.formula.api import ols

    # SKLearn Modules
    from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression
    from sklearn.feature_selection import RFE
    from sklearn.preprocessing import StandardScaler, OneHotEncoder, normalize
    from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, log_loss, confusion_matrix, RocCurveDisplay, plot_roc_curve, classification_report, accuracy_score, recall_score, precision_score, f1_score
    from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
    from sklearn.datasets import load_diabetes
    from sklearn.model_selection import train_test_split, cross_validate, KFold, cross_val_score
    from sklearn import datasets

    # Suppress future, deprecation, and SettingWithCopy warnings
    import warnings
    warnings.filterwarnings("ignore", category= FutureWarning)
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    pd.options.mode.chained_assignment = None

    # make all columns in a df viewable
    pd.options.display.max_columns = None
    pd.options.display.width = None
except ImportError:
    pass

## Section One: Data Preparation

Currently, our data is split into two csv files, fraudTrain and fraudTest. This does seem nice of the data uploader if we were going to use Test_Train_Split, however, we may want to undersample down the line, so we have to merge them.

In [4]:
# Wrapper Function to Save Time Uploading Data
if not 'fraudDF' in globals():
    if os.path.exists('data/fraudTrain.csv') and os.path.exists('data/fraudTest.csv'):
        # Load in Fraud Test and Train
        fraudTrain = pd.read_csv('data/fraudTrain.csv')
        fraudTest = pd.read_csv('data/fraudTest.csv')

        # Concatenate them together
        fraudDF = pd.concat([fraudTrain,fraudTest], axis = 0)